# Dissected and reproduced the baseline script
----


I cut the originial loader class (which takes around 6-8 hours for me to run) into separate loader functions for the different columns of the input.

I have done some small and easy parallelization (3-10x faster).

Some parts are not complete, and has to be understood in the baseline.

----

In [1]:
import os, sys,time
from multiprocessing import Pool
from multiprocessing import sharedctypes
from numpy import ctypeslib
from functools import partial

import numpy as np
import pandas as pd

from scipy.stats.mstats import mquantiles

import pysam
import pyBigWig
from Bio import SeqIO

from pyDNAbinding.binding_model import DNASequence, PWMBindingModel, DNABindingModels, load_binding_models

In [13]:
#paths
TRAIN_TSV_BASE_DIR = "/mnt/vdisk/data/synapse/encodeChallenge_Data/422/10035422/ChIPseq/labels/"
DNASE_IDR_PEAKS_BASE_DIR = "/mnt/vdisk/data/synapse/encodeChallenge_Data/251/10253251/essential_training_data/DNASE/peaks/conservative/"
DNASE_FOLD_COV_DIR = "/mnt/vdisk/data/synapse/encodeChallenge_Data/251/10253251/essential_training_data/DNASE/fold_coverage_wiggles/"
REF_GENOME="hg19.genome.fa"

#get all cell line names (imr90?)
ALL_CELL_LINES=[x.split('.')[1] for x in os.listdir(DNASE_FOLD_COV_DIR)]

#do not load all for testing
MAX_N_ROWS=int(1e5)

#use this factor now
t_factor='CTCF'

In [3]:
def my_timer(f,*args):
    """Estimate execution time for whole genome."""
    start=time.time()
    f(*args)
    print 'Estimated time for whole genome:',
    print int((time.time()-start)*56e6/MAX_N_ROWS),'s'

### The label data

In [4]:
def load_labels(t_factor,
                    nrows=MAX_N_ROWS,
                    label_dir=TRAIN_TSV_BASE_DIR):
    """Load the labels for a transcription factor for all cell lines."""
    labels_fname = label_dir + t_factor + ".train.labels.tsv.gz"
    #load the original
    temp_df=pd.read_table(labels_fname,header=0,index_col=(0,1,2),nrows=nrows)
    #change U,A,B to U,A->0 B->1!!
    temp_df[(temp_df=='U') | (temp_df=='A')]=0
    temp_df[temp_df=='B']=1
    return temp_df
    

my_timer(load_labels,t_factor)

load_labels(t_factor).tail()

Estimated time for whole genome: 182 s


A549 H1-hESC HeLa-S3 HepG2 IMR-90 K562 MCF-7
chr   start   stop                                                
chr10 5000350 5000550    0       0       0     0      0    0     0
      5000400 5000600    0       0       0     0      0    0     0
      5000450 5000650    0       0       0     0      0    0     0
      5000500 5000700    0       0       0     0      0    0     0
      5000550 5000750    0       0       0     0      0    0     0

### Load the positions which have to predicted
- could be get from the above, but if needed separately

In [14]:
def load_index(t_factor='CTCF',
               nrows=MAX_N_ROWS,
               label_dir=TRAIN_TSV_BASE_DIR):
    """Load the positions which has to predicted as an index."""
    labels_fname = label_dir + t_factor + ".train.labels.tsv.gz"
    return pd.read_table(labels_fname,header=0,index_col=(0,1,2),
                         usecols=(0,1,2),nrows=nrows).index

my_timer(load_index)

idx=load_index()

Estimated time for whole genome: 113 s


### DNASE peak data
- How will we use this data?
    - it needs to be intersected with the label indices
- todo: understand how the baseline uses it

In [6]:
def load_dnase_peaks(cell_line,
                     nrows=MAX_N_ROWS,
                     dnase_peak_dir=DNASE_IDR_PEAKS_BASE_DIR):
    """Load dnase peak data for a transcription factor and a cell line."""
    dnase_peak_fname = dnase_peak_dir + 'DNASE.' + cell_line + '.conservative.narrowPeak.gz'
    return pd.read_table(dnase_peak_fname,header=None,nrows=nrows)


my_timer(load_dnase_peaks,'A549')

load_dnase_peaks('A549').tail()

Estimated time for whole genome: 92 s


,0,1,2,3,4,5,6,7,8,9
99995,chr10,112064555,112065386,.,1000,.,17.57322,237.07674,233.09158,273
99996,chr2,198380260,198381050,.,1000,.,17.57322,131.93927,128.62138,464
99997,chr7,44887597,44888359,.,1000,.,17.57426,305.57327,301.09747,291
99998,chr2,134192461,134193257,.,1000,.,17.57469,176.14229,172.54501,601
99999,chr11,67113263,67114300,.,1000,.,17.57478,325.62643,320.99326,381


### The DNASE raw fold coverage data

- parellization made it 3x faster
    - bigwigs cannot be serialized, so i just create and close them in every process which creates an overhead.
    - probably could be faster, but there are slower stuff so it doesnt matter that much
    
- if we want to load it for many cell lines it is slower

In [7]:
#global shared result array... is there a better way?
fold_cov = sharedctypes.RawArray('d', len(idx))

def load_dnase_fold_cov(cell_line,
                        n_proc=12,
                        dnase_fold_cov_dir=DNASE_FOLD_COV_DIR ):
    """Load the raw fold coverage for a t factor and cell line."""
    dnase_fold_cov_fname = dnase_fold_cov_dir + 'DNASE.'+cell_line+'.fc.signal.bigwig'
    
    #partially apply the scorer function
    part_get_dnase_fold_cov_from_region=partial(get_dnase_fold_cov_from_region,
                                                fn=dnase_fold_cov_fname)
    #parallel execute it
    Pool(n_proc).map(part_get_dnase_fold_cov_from_region,xrange(len(idx)))

    #return a dataframe
    return pd.DataFrame({cell_line+'_dnase_fc' : fold_cov},index=idx)

def get_dnase_fold_cov_from_region(i,fn=None):
    """Get raw dnase fold coverage for a region."""
    contig,start,stop=idx[i]
    bigwig_f=pyBigWig.open(fn)
    fold_cov[i]=bigwig_f.stats(contig,start,stop)[0]
    bigwig_f.close()
    return
    

my_timer(load_dnase_fold_cov,'A549')
load_dnase_fold_cov('A549').tail()

Estimated time for whole genome: 1320 s


A549_dnase_fc
chr   start   stop                  
chr10 5000350 5000550       0.227958
      5000400 5000600       0.257754
      5000450 5000650       0.240680
      5000500 5000700       0.256136
      5000550 5000750       0.261165

### Load the motif scores

- genome need to be read only once its like 2 minutes, i do it before once

- parallelization made ~10x faster, still long time
    - genome gets copied into every process but still faster than pysam which is disk reading based

In [8]:
start=time.time()
with open(REF_GENOME, "rU") as h:
    ref_genome=SeqIO.to_dict(SeqIO.parse(h, "fasta"))
print int(time.time()-start),'s'

56 s


In [15]:
#the aggregate colnames
aggregate_region_scores_labels = ["motif_mean", "motif_max", "motif_q99",
                                  "motif_q95", "motif_q90", "motif_q75", "motif_q50"]
def aggregate_region_scores(scores,
                            quantile_probs = [0.99, 0.95, 0.90, 0.75, 0.50]):
    """Return aggregate scores of all scores from a region."""
    rv = [scores.mean()/len(scores), scores.max()]
    rv.extend(mquantiles(scores, prob=quantile_probs))
    return rv

#global shared result array... is there a better way?
shape=(len(idx),len(aggregate_region_scores_labels))
motif_scores = sharedctypes.RawArray('d', shape[0]*shape[1])

def load_motif_scores(t_factor,
                      n_proc=12):
    """Load aggregate motif scores for a t_factor for intervals in the index."""
    #load binding models
    binding_models = load_binding_models("models.yaml")
    model = binding_models.get_from_tfname(t_factor)
    
    #partially apply the scorer function
    part_get_motif_scores_from_region=partial(
        get_motif_scores_from_region,model=model)
    
    #parrallel execute it
    Pool(n_proc).map(part_get_motif_scores_from_region,xrange(len(idx)))        
    
    #shared array to numpy array
    ms=ctypeslib.as_array(motif_scores).reshape(shape)
    
    colnames=[t_factor+'_'+x for x in aggregate_region_scores_labels]
    return pd.DataFrame(ms,index=idx,columns=colnames)
    
def get_motif_scores_from_region(i,model):
    """Get motif scores from a region in index."""
    contig, start, stop = idx[i]
    #load seq from genome
    seq=str(ref_genome[contig][start:stop].seq).upper()
    #pysam is a bit slower
    #genome = pysam.FastaFile(REF_GENOME)
    #seq=genome.fetch(contig, start, stop+1).upper()
    
    #make sharred array numpy array
    ms = ctypeslib.as_array(motif_scores).reshape(shape)
    
    #get aggregate motif scores
    ms[i] = aggregate_region_scores(
        DNASequence(seq).score_binding_sites(model[0], 'MAX'))
    
    return
    
my_timer(load_motif_scores,t_factor)
load_motif_scores(t_factor).tail()

Estimated time for whole genome: 4095 s


CTCF_motif_mean  CTCF_motif_max  CTCF_motif_q99  \
chr   start    stop                                                        
chr10 50406100 50406300         6.160816        7.488257        7.340118   
      50406150 50406350         6.227698        7.340257        7.168880   
      50406200 50406400         5.868767        7.072957        7.069085   
      50406250 50406450         5.795236        7.072957        7.063795   
      50406300 50406500         5.648257        7.072957        7.039548   

                         CTCF_motif_q95  CTCF_motif_q90  CTCF_motif_q75  \
chr   start    stop                                                       
chr10 50406100 50406300        7.066297        6.943441        6.771597   
      50406150 50406350        6.996001        6.905165        6.689737   
      50406200 50406400        6.932773        6.799685        6.577077   
      50406250 50406450        6.844825        6.695881        6.481957   
      50406300 50406500        6.810221        6.687405        6.525657   

                         CTCF_motif_q50  
chr   start    stop                      
chr10 50406100 50406300        6.504157  
      50406150 50406350        6.437557  
      50406200 50406400        6.336956  
      50406250 50406450        6.283857  
      50406300 50406500        6.306056

### Put all together in one dataframe

In [12]:
#load the labels
labels_df=load_labels(t_factor)

#get the index of positions
idx=labels_df.index

#load fold coverage for all cell lines which have labels
dnase_fc_df=[ load_dnase_fold_cov(cell_line) for cell_line in labels_df.columns]

#load motif scores for the tf
motif_score_df=load_motif_scores(t_factor)

#concat tables
small_df=pd.concat(dnase_fc_df+[motif_score_df,labels_df],axis=1)

small_df.tail()

A549_dnase_fc  H1-hESC_dnase_fc  HeLa-S3_dnase_fc  \
chr   start   stop                                                         
chr10 5000350 5000550       0.227958          0.968319          0.215982   
      5000400 5000600       0.257754          0.782935          0.061195   
      5000450 5000650       0.240680          0.534555          0.000000   
      5000500 5000700       0.256136          0.255579          0.041397   
      5000550 5000750       0.261165          0.111591          0.194384   

                       HepG2_dnase_fc  IMR-90_dnase_fc  K562_dnase_fc  \
chr   start   stop                                                      
chr10 5000350 5000550        0.854929         0.111591       0.269978   
      5000400 5000600        0.496759         0.000000       0.194384   
      5000450 5000650        0.226781         0.000000       0.104391   
      5000500 5000700        0.016199         0.021598       0.014399   
      5000550 5000750        0.000000         0.111591       0.000000   

                       MCF-7_dnase_fc  CTCF_motif_mean  CTCF_motif_max  \
chr   start   stop                                                       
chr10 5000350 5000550        0.543555         6.129895        7.353956   
      5000400 5000600        0.244780         6.179895        7.353956   
      5000450 5000650        0.082793         6.283457        7.410456   
      5000500 5000700        0.000000         6.240154        7.410456   
      5000550 5000750        0.000000         6.210166        7.410456   

                       CTCF_motif_q99  ...   CTCF_motif_q90  CTCF_motif_q75  \
chr   start   stop                     ...                                    
chr10 5000350 5000550        7.180013  ...         6.910457        6.717757   
      5000400 5000600        7.278669  ...         6.927805        6.745437   
      5000450 5000650        7.343331  ...         6.992093        6.760777   
      5000500 5000700        7.281059  ...         6.999197        6.733356   
      5000550 5000750        7.281059  ...         7.004493        6.762417   

                       CTCF_motif_q50  A549 H1-hESC HeLa-S3 HepG2 IMR-90 K562  \
chr   start   stop                                                              
chr10 5000350 5000550        6.471757     0       0       0     0      0    0   
      5000400 5000600        6.472157     0       0       0     0      0    0   
      5000450 5000650        6.535056     0       0       0     0      0    0   
      5000500 5000700        6.521957     0       0       0     0      0    0   
      5000550 5000750        6.538856     0       0       0     0      0    0   

                      MCF-7  
chr   start   stop           
chr10 5000350 5000550     0  
      5000400 5000600     0  
      5000450 5000650     0  
      5000500 5000700     0  
      5000550 5000750     0  

[5 rows x 21 columns]

----

## Small training

- with training on only 1 cell line and using another  cell line for blind test

In [13]:
#lables
y_train=small_df['A549'].values.astype(int)
y_test=small_df['K562'].values.astype(int)

#dnase levels
dnase_train=small_df['A549_dnase_fc']
dnase_test=small_df['K562_dnase_fc']

#drop these
small_df.drop(['A549','K562','A549_dnase_fc','K562_dnase_fc'],
              inplace=True,axis=1)

#add the corrsponding dnase levels to all inputs
df_train=pd.DataFrame(small_df)
df_train['fold_cov']=dnase_train
df_test=pd.DataFrame(small_df)
df_test['fold_cov']=dnase_test

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

clf=RandomForestClassifier(n_estimators=500,n_jobs=12)
clf.fit(df_train,y_train)

print 'Train AUC:',roc_auc_score(y_train,clf.predict_proba(df_train)[:,1])
print 'Test AUC:',roc_auc_score(y_test,clf.predict_proba(df_test)[:,1])

Train AUC: 1.0
Test AUC: 0.956594919461
